In [1]:
from syft.core.tensor.passthrough import PassthroughTensor
from typing import Union,Optional,Any,Tuple
import numpy as np
from syft.core.tensor.config import DEFAULT_INT_NUMPY_TYPE
from syft.core.tensor.config import DEFAULT_FLOAT_NUMPY_TYPE


/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class FixedPrecisionTensor(PassthroughTensor):
    def __init__(
        self,
        value: Union[int, float, np.ndarray] = None,
        base: int = 2,
        precision: int = 16,
    ) -> None:
        self._base = base
        self._precision = precision
        self._scale = base**precision
        if value is not None:
            super().__init__(self.encode(value))
        else:
            super().__init__(None)

    def encode(self, value: Union[int, float, np.ndarray]) -> np.ndarray:
        encoded_value = np.array(self._scale * value, DEFAULT_INT_NUMPY_TYPE)
        return encoded_value

    @property
    def dtype(self) -> np.dtype:
        return getattr(self.child, "dtype", None)

    @property
    def shape(self) -> Optional[Tuple[int, ...]]:
        return getattr(self.child, "shape", None)

    def decode(self) -> Any:
        value = self.child
        scale = self._scale

        correction = (value < 0).astype(DEFAULT_INT_NUMPY_TYPE)

        dividend = np.trunc(value / scale - correction)
        remainder = value % scale
        remainder += (
            (remainder == 0).astype(DEFAULT_INT_NUMPY_TYPE) * scale * correction
        )
        value = (
            dividend.astype(DEFAULT_FLOAT_NUMPY_TYPE)
            + remainder.astype(DEFAULT_FLOAT_NUMPY_TYPE) / scale
        )
        return value

    def sanity_check(
        self, other: Union["FixedPrecisionTensor", int, float, np.ndarray]
    ) -> "FixedPrecisionTensor":
        if isinstance(other, FixedPrecisionTensor):
            if self._base != other._base or self._precision != other._precision:
                raise ValueError(
                    f"Base:{self.base,other.base} and Precision: "
                    + f"{self.precision, other.precision} should be same for "
                    + "computation on FixedPrecisionTensor"
                )
        elif isinstance(other, (int,float,np.ndarray)):
            other = FixedPrecisionTensor(
                value=other, base=self._base, precision=self._precision
            )
        else:
            raise ValueError(f"Invalid type for FixedPrecisionTensor: {type(other)}")

        return other

    def __add__(self, other: Any):
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child + other.child
        return res

    def __sub__(self, other: Any):
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child - other.child
        return res

    def __mul__(self, other: Any): 
        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        other = self.sanity_check(other)
        res.child = self.child * other.child
        res = res / self._scale
        return res

    def __lt__(self, other: Any):
        other = self.sanity_check(other)
        value = (self.child < other.child) * 1

        res = FixedPrecisionTensor(
            value=value, base=self._base, precision=self._precision
        )
        return res

    def __gt__(self, other: Any):
        other = self.sanity_check(other)
        value = (self.child > other.child) * 1
        res = FixedPrecisionTensor(
            value=value, base=self._base, precision=self._precision
        )
        return res

    def __truediv__(
        self, other: Union[int, np.integer, "FixedPrecisionTensor"]
    ) -> "FixedPrecisionTensor":
        if isinstance(other, FixedPrecisionTensor):
            raise ValueError("We do not support Private Division yet.")

        res = FixedPrecisionTensor(base=self._base, precision=self._precision)
        if isinstance(self.child, np.ndarray) or np.isscalar(self.child):
            res.child = np.trunc(self.child / other).astype(DEFAULT_INT_NUMPY_TYPE)
        else:
            res.child = self.child / other
        return res

        

In [4]:
def sign(data):
    return (data>0) + (data<0)*-1

def modulus(data):
    return sign(data) * data

def exp(value, iterations = 8):
    result = (value / 2**iterations) + 1
    for _ in range(iterations):
        result = result * result
    return result

def log(data, iterations = 2, exp_iterations = 8):
    y = data / 31 + 1.59 - 20 * exp((-2 * data - 1.4), iterations=exp_iterations)

    # 6th order Householder iterations
    for i in range(iterations):
        h = [1 - data * exp((-1 * y), iterations=exp_iterations)]
        for j in range(1, 5):
            h.append(h[-1] * h[0])

        y -= h[0] * (1 + h[0] / 2 + h[1] / 3 + h[2] / 4 + h[3] / 5 + h[4] / 6)

    return y


def reciprocal(data, method = "NR", nr_iters = 10):
    method = method.lower()
    if method == "nr":
        new_data = modulus(data)
        result = exp(new_data * -1 + 0.5) * 3 + 0.003
        for i in range(nr_iters):
            result = result * 2 - result * result * new_data
        return result * sign(data)
    elif method == "log":
        new_data = modulus(data)
        return exp(-1 * log(new_data)) * sign(data)
    else:
        raise ValueError(f"Invalid method {method} given for reciprocal function")




In [55]:
val = np.array([-2154,-3154,-5614,-7518])


In [56]:
a = FixedPrecisionTensor(val)

In [57]:
a

FixedPrecisionTensor(child=[-141164544 -206700544 -367919104 -492699648])

In [58]:
a.decode()

array([-2154., -3154., -5614., -7518.])

In [59]:
reciprocal(a).decode()

array([-1.13315451e+07, -1.17961730e+09,  1.49793323e+09, -3.33832374e+07])

In [40]:
1/val

array([ 0.00046425, -0.00031706,  0.00017813, -0.00013301])

In [46]:
new_data = modulus(a)
# result = exp(new_data * -1 + 0.5) * 3 + 0.003
result = 3 * exp((2 * new_data-1)*-1) + 0.003

In [50]:
2**2154

2621867017208110857291278646274757833211428251255572119667240856128689859399967597712701449540498702686528012029570221268104406033856895404835653389828332998407065448122619523201130842673994127983631115031360198512870417510041732087342228812898057254663393396381482407931849193438026565408952286452216349898494859674247283552617173642549493707906614081953874823532621970844112340813968948171522563474938937617953781459396078648018289743901613088748540468750755478262312096444327164053458687952496362375276274447747448345518292599380467806650757224411640931683375968501388566930703594293583757984396572562873200458373170942241638564389578408653225984

-2154